In [363]:
import pandas as pd
import os 
import numpy as np
from collections import Counter

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC


from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

os.chdir('C:/Users/George/Documents/Baseball/Pitch Prediction')

In [147]:
rawData = pd.read_csv('PitcherData15-19.csv')
rawData = rawData.drop(['Unnamed: 0'], axis=1)

C:\Users\George\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [250]:
fastballs = ['FF', 'FT', 'FC', 'FA']

slider = ['SL', 'SC']

changeup = ['CH', 'EP']

sinker = ['FO', 'SI', 'FS']

curveball = ['KC', 'CU']

knuckleball = ['KN']

pitchout = ['PO']

intentional = ['IN']

unknown = ['UN']

In [251]:
conditions = [
    rawData['pitch_type'].isin(fastballs),
    rawData['pitch_type'].isin(slider),
    rawData['pitch_type'].isin(changeup),
    rawData['pitch_type'].isin(sinker),
    rawData['pitch_type'].isin(curveball),
    rawData['pitch_type'].isin(knuckleball),
    rawData['pitch_type'].isin(pitchout),
    rawData['pitch_type'].isin(intentional),
    rawData['pitch_type'].isin(unknown),
]

status = ['fastball', 'slider', 'changeup', 'sinker', 'curveball', 
          'knuckleball', 'pitchout', 'intentional', 'unknown']

rawData['pitch_type_trunk'] = np.select(conditions, status, default='fastball')

In [274]:
randomPitchers = list(pd.Series(rawData['pitcher'].unique()).sample(10))

In [275]:
data = rawData[rawData['pitcher'].isin(randomPitchers)]

In [463]:
data = rawData[rawData['player_name'] == 'Corey Kluber']

In [464]:
data.player_name.unique()

array(['Corey Kluber'], dtype=object)

In [465]:
#Binary on base not on a base
data[['on_3b', 'on_2b', 'on_1b']] = data[['on_3b', 'on_2b', 'on_1b']].isna() == False

C:\Users\George\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [466]:
#adding indices 
data['game_date_index'] = data['game_date'].apply(lambda a: a.replace('-', ''))
data['game_index'] = data['game_date_index'] + '_' + data.away_team + '_' + data.home_team 
data['pitcher_apperance_index'] = data['game_date_index'] + '_' + data['pitcher'].astype(str)    
data['atbat_id'] = data['pitcher_apperance_index'] + "_" + data['at_bat_number'].astype(str)

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [467]:
hit_list = ['double', 'single', 'home_run', 'triple']
data2 = pd.DataFrame()
for i in data['pitcher_apperance_index'].unique():
    
    #selecing single game
    apperance = data[data['pitcher_apperance_index'] == i] 
    apperance = apperance.sort_values(['at_bat_number','pitch_number']) #sorting at-bats and pitches
    
    #pitch count
    apperance['pitch_count_game'] = np.arange(len(apperance)) 
    
    #Number of walks
    apperance['number_of_walks'] = (apperance['events'] == 'walk').cumsum().shift(1).fillna(0)
    
    #Number of hits
    apperance['number_of_hits'] = apperance['events'].isin(hit_list).cumsum().shift(1).fillna(0)

    #Prior Pitch
    apperance['last_pitch'] = apperance['pitch_type_trunk'].shift(1).fillna('NewAB')
    
    #Prior Event
    apperance['last_event'] = apperance['description'].shift(1).fillna('FirstAB')


    data2 = data2.append(apperance)

In [468]:
hit = ['hit_into_play','hit_into_play_no_out', 'hit_into_play_score', 'pitchout_hit_into_play_score']

strike = ['swinging_strike', 'foul', 'called_strike', 'swinging_strike_blocked',
       'foul_tip', 'foul_bunt', 'missed_bunt', 'swinging_pitchout', 'foul_pitchout', 'bunt_foul_tip']

ball = ['ball', 'blocked_ball', 'intent_ball', 'hit_by_pitch', 'automatic_ball']

In [469]:
data2['last_event'] = np.where(data2['at_bat_number']==1, 'FirstAB', data2['last_event'])

In [470]:
conditions = [
    (data2['on_1b'] == True) & (data2['on_2b'] == True) & (data2['on_3b'] == True),
    (data2['on_2b'] == True) | (data2['on_3b'] == True),
    (data2['on_1b'] == True)
]

status = ['BL', 'RISP', '1B']

data2['Runners_On'] = np.select(conditions, status, default='BasesEmpty')

In [471]:
data2['Fav_Pitcher_Matchup'] = data2['p_throws'] == data2['stand']

In [472]:
data2.last_event[data2['last_event'].isin(strike)] = 'strike'

data2.last_event[data2['last_event'].isin(ball)] = 'ball'

data2.last_event[data2['last_event'].isin(hit)] = 'hit'

C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\George\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [473]:
data2['run_diff'] = data2.fld_score - data2.bat_score

In [474]:
data2['if_fielding_alignment'] = data2.if_fielding_alignment.fillna('Standard')

In [475]:
toOneHot = np.array(data2[['Runners_On', 'last_pitch', 'last_event', 'if_fielding_alignment']])
onehotencoder = OneHotEncoder(sparse=False) 
oneHotData = onehotencoder.fit_transform(toOneHot)

In [476]:
oneHotData = pd.DataFrame(oneHotData, columns=onehotencoder.get_feature_names(['Runners_On', 'last_pitch', 
                                                                               'last_event', 'if_fielding_alignment']))
data2 = data2.reset_index(drop=True)

In [477]:
data2 = pd.concat([data2, oneHotData], axis=1)

In [478]:
data2['pitch_type_trunk'].fillna('fastball', inplace = True)

In [479]:
# Label encoding Pitch Type so can be fed into models
labelencoder = LabelEncoder()
PitchTypeEncoded = labelencoder.fit_transform(data2['pitch_type_trunk'])

data2['pitch_type_trunk_le'] = PitchTypeEncoded

### Testing on different pitchers

In [480]:
#selected_pitcher = randomPitchers[8]
#single_pitcher = data2[data2['pitcher'] == selected_pitcher]
single_pitcher = data2
print(single_pitcher['player_name'].unique())
single_pitcher.pitch_type_trunk.value_counts()

['Corey Kluber']


sinker         4131
fastball       3865
curveball      2739
slider         1612
changeup        730
intentional      14
pitchout          1
Name: pitch_type_trunk, dtype: int64

In [481]:
pitch_arsenal = pd.DataFrame(single_pitcher.pitch_type_trunk.value_counts() / len(single_pitcher)).reset_index()
pitch_arsenal.columns = ['pitch', 'pitch_fequency']
pitchesToDrop = pitch_arsenal.loc[pitch_arsenal['pitch_fequency'] < 0.03, 'pitch'].values

In [482]:
pitch_columns = lambda a : "last_pitch_{}".format(a)

last_pitch_columns = pd.Series(single_pitcher['pitch_type_trunk'].unique()).apply(pitch_columns)
last_pitch_columns = list(last_pitch_columns)

In [483]:
StandardPredVars = ['strikes', 'balls','pitch_count_game','Runners_On_RISP', 'Fav_Pitcher_Matchup',
                    'run_diff', 'if_fielding_alignment_Strategic']
PredVars = StandardPredVars + last_pitch_columns

In [484]:
single_pitcher.drop(single_pitcher[single_pitcher['pitch_type_trunk'].isin(pitchesToDrop)].index , inplace=True)

In [485]:
X = single_pitcher[PredVars]
y = single_pitcher['pitch_type_trunk_le'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)

In [486]:
dt_clf = DecisionTreeClassifier(min_samples_split=2,
                                max_depth=4,
                                random_state=23)
dt_clf.fit(X_train, y_train)
preds = dt_clf.predict(X_test)
acc = (preds == y_test).mean()
print('Accuracy is {}%'.format(round(acc*100,1)))

Accuracy is 33.2%


In [487]:
print(classification_report_imbalanced(y_test, preds))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00       179
          1       0.36      0.29      0.86      0.32      0.50      0.24       674
          2       0.32      0.38      0.63      0.35      0.49      0.23      1011
          5       0.34      0.50      0.56      0.40      0.53      0.28      1007
          6       0.00      0.00      1.00      0.00      0.00      0.00       399

avg / total       0.27      0.33      0.72      0.30      0.42      0.21      3270



C:\Users\George\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Trying out Imbalance Learn package

In [488]:
print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

Training target statistics: Counter({5: 3124, 2: 2854, 1: 2065, 6: 1213, 0: 551})
Testing target statistics: Counter({2: 1011, 5: 1007, 1: 674, 6: 399, 0: 179})


### Random Oversampled SVC 

In [489]:
pipe = make_pipeline(RandomOverSampler(random_state=23), SVC())
pipe.fit(X_train, y_train)
print(classification_report_imbalanced(y_test, pipe.predict(X_test)))

C:\Users\George\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


                   pre       rec       spe        f1       geo       iba       sup

          0       0.12      0.50      0.79      0.19      0.63      0.38       179
          1       0.30      0.39      0.76      0.34      0.55      0.29       674
          2       0.35      0.18      0.85      0.24      0.39      0.14      1011
          5       0.39      0.20      0.86      0.27      0.42      0.16      1007
          6       0.16      0.25      0.82      0.20      0.45      0.19       399

avg / total       0.32      0.26      0.83      0.26      0.45      0.20      3270



### SVM SMOTE Oversampling

In [490]:
from imblearn.over_sampling import SVMSMOTE

In [491]:
sm = SVMSMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(np.array(X_train), y_train.ravel()) 

In [492]:
smote_clf = SVC(random_state=23)

smote_clf.fit(X_train_res, y_train_res)
preds = smote_clf.predict(X_test)

C:\Users\George\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [493]:
print(classification_report_imbalanced(y_test, preds))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.14      0.26      0.90      0.18      0.49      0.22       179
          1       0.31      0.43      0.75      0.36      0.57      0.31       674
          2       0.35      0.27      0.78      0.30      0.45      0.20      1011
          5       0.37      0.27      0.80      0.31      0.46      0.20      1007
          6       0.18      0.22      0.86      0.20      0.43      0.18       399

avg / total       0.31      0.29      0.79      0.30      0.48      0.22      3270



In [494]:
(preds == y_test).mean()

0.29388379204892967

In [495]:
pd.Series(preds).value_counts() / len(preds)

1    0.286544
2    0.237920
5    0.223547
6    0.147401
0    0.104587
dtype: float64

In [496]:
pd.Series(y_test).value_counts() / len(y_test)

2    0.309174
5    0.307951
1    0.206116
6    0.122018
0    0.054740
Name: pitch_type_trunk_le, dtype: float64